<a href="https://colab.research.google.com/github/Mmlirani/Pong-Pixels-Reinforcement-Learning/blob/master/cityflow3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y build-essential cmake python3-dev libboost-all-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,601 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,450 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,425 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,654 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 15.5 MB in 1s (10.4 MB/s)
R

In [2]:
!git clone https://github.com/cityflow-project/CityFlow.git


Cloning into 'CityFlow'...
remote: Enumerating objects: 780, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 780 (delta 1), reused 2 (delta 0), pack-reused 772 (from 1)
Receiving objects: 100% (780/780), 334.65 KiB | 3.31 MiB/s, done.
Resolving deltas: 100% (448/448), done.


In [3]:
%cd CityFlow
!mkdir build
%cd build
!cmake ..
!make -j4


/content/CityFlow
/content/CityFlow/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Submodule update, this may take some time...
Submodule 'extern/pybind11' (https://github.com/cityflow-project/pybind11.git) registered for path 'extern/pybind11'
Submodule 'extern/rapidjson' (https://github.com/cityflow-project/rapidjson.git) registered for path 'extern/rapidjson'
Cloning into '/content/CityFlow/extern/pybind11'...
Cloning into '/content/CityFlow/extern/rapidjson'...
Submodule path 'extern/pybind11': checked out 'a8ee79d08e9705e2

In [4]:
%cd ..
!pip install .


/content/CityFlow
Processing /content/CityFlow
  Preparing metadata (setup.py) ... done
  Created wheel for CityFlow: filename=CityFlow-0.1-cp310-cp310-linux_x86_64.whl size=247984 sha256=8942781efb33d88216c36e11c86bcbc47e3ddb6b24a54d5270aaaef8e1c69cba
  Stored in directory: /tmp/pip-ephem-wheel-cache-bbsv0oud/wheels/69/3d/03/c6c15672e35ee6e4b000b42b6012bb9fc6d03bbf1fa5d8df6f
Successfully built CityFlow


In [5]:
import cityflow
print("CityFlow installed successfully!")


CityFlow installed successfully!


In [6]:
import cityflow
import pandas as pd
import os
import json
import math
import numpy as np
import itertools



class CityFlowEnvM(object):
    '''
    multi inersection cityflow environment
    '''

    def __init__(self,
                 lane_phase_info,
                 intersection_id,
                 num_step=2000,
                 thread_num=1,
                 cityflow_config_file='example/config_1x2.json',
                 path_to_log='result',
                 dataset = '1_3',
                 ):
        self.eng = cityflow.Engine(cityflow_config_file, thread_num=thread_num)
        self.num_step = num_step
        self.intersection_id = intersection_id  # list, [intersection_id, ...]
        self.state_size = None
        self.lane_phase_info = lane_phase_info  # "intersection_1_1"

        self.path_to_log = path_to_log
        self.current_phase = {}
        self.current_phase_time = {}
        self.start_lane = {}
        self.end_lane = {}
        self.phase_list = {}
        self.lane_mapping = {}
        self.phase_startLane_mapping = {}
        self.intersection_lane_mapping = {}  # {id_:[lanes]}
        self.dataset = dataset
        self.lane_intensity = {}

        initial_phase = {}
        for id_ in self.intersection_id:
            self.start_lane[id_] = self.lane_phase_info[id_]['start_lane']
            self.end_lane[id_] = self.lane_phase_info[id_]['end_lane']

            # print(self.start_lane[id_])
            # print(self.end_lane[id_])
            # print("\n")

            self.lane_mapping[id_] = self.lane_phase_info[id_]["lane_mapping"]
            # print(self.lane_mapping[id_])
            self.phase_startLane_mapping[id_] = self.lane_phase_info[id_]["phase_startLane_mapping"]

            self.phase_list[id_] = self.lane_phase_info[id_]["phase"]
            self.current_phase[id_] = self.phase_list[id_][0]
            self.current_phase_time[id_] = 0
            initial_phase[id_] = 1
            self.lane_intensity[id_] = {}
            self.lane_intensity[id_]['start'] = [0 for _ in self.start_lane[id_]]
            self.lane_intensity[id_]['end'] = [0 for _ in self.end_lane[id_]]
        self.get_state(initial_phase)  # set self.state_size

    def reset(self):
        self.eng.reset()

    def step(self, action_phase, cur_step):
        '''
        action: {intersection_id: phase, ...}
        '''
        for id_, a in action_phase.items():
            if self.current_phase[id_] == a:
                self.current_phase_time[id_] += 1
            else:
                self.current_phase[id_] = a
                self.current_phase_time[id_] = 1
            self.eng.set_tl_phase(id_, self.current_phase[id_])  # set phase of traffic light
        self.eng.next_step()
        if cur_step % 5 == 4:
            distances = self.eng.get_vehicle_distance()
            for id_ in self.intersection_id:
                self.lane_intensity[id_]['start'] = [self.get_lanepressure(id_, lane, distances) for lane in
                                                     self.start_lane[id_]]
                self.lane_intensity[id_]['end'] = [self.get_lanepressure(id_, lane, distances) for lane in
                                                   self.end_lane[id_]]
        return self.get_state(action_phase), self.get_reward()

    def get_state(self, action_phase = None):
        # print(action_phase)
        state = {id_: self.get_state_(id_, action_phase[id_]) for id_ in self.intersection_id}
        return state

    # get lane pressure with each vehicle
    def get_lanepressure(self, id_, lane, distances):
        L = 300
        if self.dataset == 'jinan':
            if lane[-3] == '0' or lane[-3] == '2':
                L = 400
            else:
                L = 800
        if self.dataset == 'hangzhou':
            if lane[-3] == '0' or lane[-3] == '2':
                L = 800
            else:
                L = 600
        lane_pressure = 0
        sigma = 1.5
        max_speed = 11.111
        vehicles = self.eng.get_lane_vehicles()[lane]
        # print("vehicles:",vehicles)
        if lane in self.start_lane[id_]:
            for v in vehicles:
                x = distances[v]
                lane_pressure += float(format(math.log(
                x / L * sigma * (max_speed - float(self.eng.get_vehicle_info(v)["speed"])) / (
                        float(self.eng.get_vehicle_info(v)["speed"]) + 1) + 1), '.4f'))
                # lane_pressure += float(format(math.log(
                # sigma * (max_speed - float(self.eng.get_vehicle_info(v)["speed"])) / (
                #         float(self.eng.get_vehicle_info(v)["speed"]) + 1) + 1), '.4f'))
                # print("speed for", v, ":", self.eng.get_vehicle_info(v)["speed"],"pressure:",format(math.log(w1*float(self.eng.get_vehicle_info(v)["speed"]) + w2*max_speed + 1),'.4f'))
        else:
            for v in vehicles:
                x = distances[v]
                lane_pressure += float(format(math.log(
                (L - x) / L * sigma * (max_speed - float(self.eng.get_vehicle_info(v)["speed"])) / (
                        float(self.eng.get_vehicle_info(v)["speed"]) + 1) + 1), '.4f'))
                # lane_pressure += float(format(math.log(
                # sigma * (max_speed - float(self.eng.get_vehicle_info(v)["speed"])) / (
                #         float(self.eng.get_vehicle_info(v)["speed"]) + 1) + 1), '.4f'))
                # print("speed for", v, ":", self.eng.get_vehicle_info(v)["speed"],"pressure:",format(math.log(w1*float(self.eng.get_vehicle_info(v)["speed"]) + w2*max_speed + 1),'.4f'))
        return lane_pressure

    def get_neigh_pressure(self, nei_id_, row, col, nei_row, nei_col, action_phase = None):
        pressure = 0
        start_vehicle_count = self.lane_intensity[nei_id_]['start']

        if nei_row < row: # left, pressure comes from turn-right(start_vehicle_count[5]) AND (WE(start_vehicle_count[1]) OR NSL(start_vehicle_count[6]))
            if action_phase == 1: # WE
                pressure += start_vehicle_count[5] + start_vehicle_count[1]
            if action_phase == 4:  # NSL
                pressure += start_vehicle_count[5] + start_vehicle_count[6]

        if nei_row > row: # right, pressure comes from turn-right(start_vehicle_count[8]) AND (WE(start_vehicle_count[10]) OR NSL(start_vehicle_count[3]))
            if action_phase == 1: # WE
                pressure += start_vehicle_count[8] + start_vehicle_count[10]
            if action_phase == 4:  # NSL
                pressure += start_vehicle_count[8] + start_vehicle_count[3]

        if nei_col < col: # above, pressure comes from turn-right(start_vehicle_count[11]) AND (NS(start_vehicle_count[4]) OR WEL(start_vehicle_count[0]))
            if action_phase == 2: # NS
                pressure += start_vehicle_count[11] + start_vehicle_count[4]
            if action_phase == 3:  # WEL
                pressure += start_vehicle_count[11] + start_vehicle_count[0]

        if nei_col > col: # below, pressure comes from turn-right(start_vehicle_count[2]) AND (NS(start_vehicle_count[7]) OR WEL(start_vehicle_count[9]))
            if action_phase == 2: # NS
                pressure += start_vehicle_count[2] + start_vehicle_count[7]
            if action_phase == 3:  # WEL
                pressure += start_vehicle_count[2] + start_vehicle_count[9]

        return pressure

    def get_state_(self, id_, action_phase = None):
        inters = str.split(id_, '_')
        row = int(inters[1])
        column = int(inters[2])
        neighbor = [] # neighbor intersection
        neighbor.append([row - 1, column])
        neighbor.append([row, column - 1])
        neighbor.append([row + 1, column])
        neighbor.append([row, column + 1])
        eta = 0.1 # discount ctor of neighbour's pressure

        state = self.intersection_info(id_)
        intensity = []
        temp = []

        # 对应车道求压力差
        end_vehicle_count_avg = []
        for i in range(4):
            end_vehicle_count_avg.append(
                math.ceil(sum([self.lane_intensity[id_]['end'][j] for j in range(i * 3, i * 3 + 3)]) / 3))
        # print(start_vehicle_count)
        # print(end_vehicle_count)
        start_vehicle_count_cop = []
        index = [1, 0, 2, 1, 0, 3, 3, 2]
        for i in range(len(self.lane_intensity[id_]['start'])):
            if i % 3 != 2:
                start_vehicle_count_cop.append(self.lane_intensity[id_]['start'][i])
        # print(start_vehicle_count_cop)
        # print(end_vehicle_count_avg)
        for i in range(len(start_vehicle_count_cop)):
            temp.append(start_vehicle_count_cop[i] - end_vehicle_count_avg[index[i]])

        intensity.append(temp[1] + temp[7])  # straight
        intensity.append(temp[3] + temp[5]) # straight
        intensity.append(temp[0] + temp[6]) # left
        intensity.append(temp[2] + temp[4]) # left

        return_state = intensity + [state['current_phase']] # add id_'s current_phase

        for e in neighbor: # add neghbors' current_phase
            interid = inters[0] + '_' + str(e[0]) + '_' + str(e[1])
            if interid in self.intersection_id:
                return_state.append(eta * self.get_neigh_pressure(nei_id_=interid, row=row, col=column, nei_col=e[0], nei_row=e[1], action_phase=action_phase))
            else:
                return_state.append(0)

        return self.preprocess_state(return_state)

    def get_timing_(self, id_, phase):
        state = self.intersection_info(id_)
        start_vehicle_count = [state['start_lane_vehicle_count'][lane] for lane in self.start_lane[id_]]
        end_vehicle_count = [state['end_lane_vehicle_count'][lane] for lane in self.end_lane[id_]]
        start_vehicle_count_cop = []  # 1->1,7; 2->3,5; 3->0,6; 4->2,4
        for i in range(len(start_vehicle_count)):
            if i % 3 != 2:
                start_vehicle_count_cop.append(start_vehicle_count[i])

        phase_lane = [[1,7], [3,5], [0,6], [2,4]]
        w1 = 1
        w2 = 2
        max_count = max(start_vehicle_count_cop[phase_lane[phase-1][0]], start_vehicle_count_cop[phase_lane[phase-1][1]])
        min_count = min(start_vehicle_count_cop[phase_lane[phase-1][0]], start_vehicle_count_cop[phase_lane[phase-1][1]])
        vehicle_count = (min_count * w1 + max_count * w2)/(w1+w2)
        timing = math.ceil(vehicle_count/2)*5
        if timing>25:
            timing = 25
        if timing < 5:
            timing = 5
        return vehicle_count, timing

    def intersection_info(self, id_):
        '''
        info of intersection 'id_'
        '''
        state = {}
        state['lane_vehicle_count'] = self.eng.get_lane_vehicle_count()
        state['lane_waiting_vehicle_count'] = self.eng.get_lane_waiting_vehicle_count()
        state['lane_vehicles'] = self.eng.get_lane_vehicles()
        state['vehicle_speed'] = self.eng.get_vehicle_speed()
        state['start_lane_vehicle_count'] = {lane: state['lane_vehicle_count'][lane] for lane in self.start_lane[id_]}
        state['end_lane_vehicle_count'] = {lane: state['lane_vehicle_count'][lane] for lane in self.end_lane[id_]}
        state['start_lane_waiting_vehicle_count'] = {lane: state['lane_waiting_vehicle_count'][lane] for lane in
                                                     self.start_lane[id_]}
        state['end_lane_waiting_vehicle_count'] = {lane: state['lane_waiting_vehicle_count'][lane] for lane in
                                                   self.end_lane[id_]}
        state['start_lane_vehicles'] = {lane: state['lane_vehicles'][lane] for lane in self.start_lane[id_]}
        state['end_lane_vehicles'] = {lane: state['lane_vehicles'][lane] for lane in self.end_lane[id_]}
        state['start_lane_speed'] = {
            lane: np.sum(list(map(lambda vehicle: state['vehicle_speed'][vehicle], state['lane_vehicles'][lane]))) / (
                    state['lane_vehicle_count'][lane] + 1e-5) for lane in
            self.start_lane[id_]}  # compute start lane mean speed
        state['end_lane_speed'] = {
            lane: np.sum(list(map(lambda vehicle: state['vehicle_speed'][vehicle], state['lane_vehicles'][lane]))) / (
                    state['lane_vehicle_count'][lane] + 1e-5) for lane in
            self.end_lane[id_]}  # compute end lane mean speed
        state['current_phase'] = self.current_phase[id_]
        state['current_phase_time'] = self.current_phase_time[id_]
        return state

    def preprocess_state(self, state, action = None):
        return_state = np.array(state)
        if self.state_size is None:
            self.state_size = len(return_state.flatten())
        if action != None:
            return_state = np.reshape(return_state, [1, self.state_size+1])
        else:
            return_state = np.reshape(return_state, [1, self.state_size])

        return return_state

    def get_reward(self):
        reward = {id_: self.get_reward_(id_) for id_ in self.intersection_id}
        return reward

    def get_reward_(self, id_):
        '''
        every agent/intersection's reward
        '''
        start_vehicle_count = self.lane_intensity[id_]['start']
        end_vehicle_count = self.lane_intensity[id_]['end']
        start_vehicle_count_cop = []
        end_vehicle_count_cop = []
        for i in range(len(start_vehicle_count)):
            if i % 3 != 2:
                start_vehicle_count_cop.append(start_vehicle_count[i])
                end_vehicle_count_cop.append(end_vehicle_count[i])
        intensity = sum(start_vehicle_count_cop) - sum(end_vehicle_count_cop)

        reward = -intensity
        return reward

    def get_pressure(self):
        pressure = {id_: self.get_pressure_(id_) for id_ in self.intersection_id}
        return pressure

    def get_pressure_(self, id_):
        '''
        every agent/intersection's reward
        '''
        state = self.intersection_info(id_)
        start_vehicle_count = [state['start_lane_vehicle_count'][lane] for lane in self.start_lane[id_]]
        end_vehicle_count = [state['end_lane_vehicle_count'][lane] for lane in self.end_lane[id_]]
        pressure = sum(start_vehicle_count) - sum(end_vehicle_count)
        return pressure

    def get_score(self):
        score = {id_: self.get_score_(id_) for id_ in self.intersection_id}
        return score

    def get_score_(self, id_):
        state = self.intersection_info(id_)
        start_lane_waiting_vehicle_count = state['start_lane_waiting_vehicle_count']
        end_lane_waiting_vehicle_count = state['end_lane_waiting_vehicle_count']
        score = -1 * np.sum(
            list(start_lane_waiting_vehicle_count.values()) + list(end_lane_waiting_vehicle_count.values()))
        return score

    def bulk_log(self):
        # self.eng.print_log(os.path.join(self.path_to_log, "replay.txt"))
        self.eng.set_replay_file((os.path.join(self.path_to_log, "replay.txt")))

In [7]:
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


class DQNAgent(object):
    def __init__(self,
                 intersection_id,
                 state_size=9,
                 action_size=8,
                 batch_size=32,
                 phase_list=[],
                 timing_list = [],
                 env=None
                 ):
        self.env = env
        self.intersection_id = intersection_id
        self.state_size = state_size
        self.action_size = action_size
        self.memory_size = 2000
        self.memory = deque(maxlen=self.memory_size)
        self.latest_state = np.array([0 for _ in range(9)])
        self.gamma = 0.95  # discount rate
        self.epsilon = 0.05  # exploration rate
        self.learning_rate = 0.001
        self.step = 0
        self.batch_size = batch_size
        self.model = self._build_model()

        self.phase_list = phase_list
        self.timing_list = timing_list

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(100, input_dim=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state):
        action = self.phase_list.index(action)  # index
        self.memory.append((state, action, reward, next_state))
        self.latest_state = next_state

    def remember_timing(self, state, timing, reward, next_state):
        timing = self.timing_list.index(timing)  # index
        self.memory.append((state, timing, reward, next_state))

    def choose_action(self, state):
        if np.random.uniform() < self.epsilon - self.step * 0.0002:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self):  # Timing if flag for agent_timing
        if len(self.memory) < self.batch_size:
            return
        self.step += 1
        state_t = self.memory[-1][-1]
        replay_batch = random.sample(self.memory, self.batch_size)
        s_batch = np.reshape(np.array([replay[0] for replay in replay_batch]), [self.batch_size, self.state_size])
        next_s_batch = np.reshape(np.array([replay[3] for replay in replay_batch]),
                                  [self.batch_size, self.state_size])

        Q = self.model.predict(s_batch)
        Q_next = self.model.predict(next_s_batch)

        learning_rate = 1
        for i, replay in enumerate(replay_batch):
            _, a, reward, state_n = replay
            if (state_t == state_n).all():
                Q[i][a] = (1 - learning_rate) * Q[i][a] + learning_rate * reward
            else:
                Q[i][a] = (1 - learning_rate) * Q[i][a] + learning_rate * (reward + self.gamma * np.amax(Q_next[i]))

        # 传入网络训练
        # print("s_batch:\n", s_batch, "Q:\n", Q)
        self.model.fit(s_batch, Q, verbose=0)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
        # print("model saved:{}".format(name))


class MDQNAgent(object):
    def __init__(self,
                 intersection,
                 state_size=17,
                 batch_size=32,
                 phase_list={},
                 env=None):

        self.intersection = intersection
        self.agents = {}
        self.make_agents(intersection, state_size, batch_size, phase_list, env)

    def make_agents(self, intersection, state_size, batch_size, phase_list, env):
        for id_ in self.intersection:
            self.agents[id_] = DQNAgent(id_,
                                        state_size=state_size,
                                        action_size=len(phase_list[id_]),
                                        batch_size=batch_size,
                                        phase_list=phase_list[id_],
                                        env=env
                                        )

    def remember(self, state, action, reward, next_state):
        for id_ in self.intersection:
            self.agents[id_].remember(state[id_],
                                      action[id_],
                                      reward[id_],
                                      next_state[id_])

    def choose_action(self, state):
        action = {}
        for id_ in self.intersection:
            action[id_] = self.agents[id_].choose_action(state[id_])
        return action

    def replay(self):
        for id_ in self.intersection:
            self.agents[id_].replay()

    def avg_model_weights(self):
        weights = []
        for id_ in self.intersection:
            weights.append(self.agents[id_].model.get_weights())
        avg_weight = np.average(weights, axis=0)

        for id_ in self.intersection:
            self.agents[id_].model.set_weights(avg_weight)

    def load(self, name):
        for id_ in self.intersection:
            assert os.path.exists(name + '.' + id_), "Wrong checkpoint, file not exists!"
            self.agents[id_].load(name + '.' + id_)

    def save(self, name):
        for id_ in self.intersection:
            self.agents[id_].save(id_ + '.' + name)


In [8]:
import json
import argparse
import numpy as np


def parse_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument("--scenario", type=str, default="hangzhou_1x1_bc-tyc_18041607_1h")
    parser.add_argument("--num_step", type=int, default=3600)
    return parser.parse_args()


def parse_roadnet(roadnetFile):
    roadnet = json.load(open(roadnetFile))
    lane_phase_info_dict = {}

    # many intersections exist in the roadnet and virtual intersection is controlled by signal
    for intersection in roadnet["intersections"]:
        if intersection['virtual']:
            continue
        lane_phase_info_dict[intersection['id']] = {"start_lane": [],
                                                    "end_lane": [],
                                                    "phase": [],
                                                    "lane_mapping": {},
                                                    "phase_startLane_mapping": {},
                                                    "phase_roadLink_mapping": {}}
        road_links = intersection["roadLinks"]

        start_lane = []
        end_lane = []
        roadLink_lane_pair = {}
        # roadLink_lane_pair = {ri: [] for ri in
        #                       range(len(road_links))}  # roadLink includes some lane_pair: (start_lane, end_lane)
        rik = 0
        for r in road_links:
            if r['type'] != 'turn_right':
                roadLink_lane_pair[rik] = []
                rik += 1

        rik = 0
        for ri in range(len(road_links)):
            road_link = road_links[ri]
            for lane_link in road_link["laneLinks"]:
                sl = road_link['startRoad'] + "_" + str(lane_link["startLaneIndex"])
                el = road_link['endRoad'] + "_" + str(lane_link["endLaneIndex"])
                start_lane.append(sl)
                end_lane.append(el)
                # roadLink_lane_pair[ri].append((sl, el))
                if road_link['type'] != 'turn_right':
                    roadLink_lane_pair[rik].append([sl, el])

            if road_link['type'] != 'turn_right':
                roadLink_lane_pair[rik].append(road_link['direction'])
                rik += 1

        lane_phase_info_dict[intersection['id']]["start_lane"] = sorted(list(set(start_lane)))
        lane_phase_info_dict[intersection['id']]["end_lane"] = sorted(list(set(end_lane)))
        lane_phase_info_dict[intersection['id']]["lane_mapping"] = roadLink_lane_pair

        # for phase_i in range(1, len(intersection["trafficLight"]["lightphases"])):
        for phase_i in range(1, 5):#PressLight 4相位
            # p = intersection["trafficLight"]["lightphases"][phase_i]
            # lane_pair = []
            # start_lane = []
            # for ri in p["availableRoadLinks"]:
            #     lane_pair.extend(roadLink_lane_pair[ri])
            #     if roadLink_lane_pair[ri][0][0] not in start_lane:
            #         start_lane.append(roadLink_lane_pair[ri][0][0])
            lane_phase_info_dict[intersection['id']]["phase"].append(phase_i)
            # lane_phase_info_dict[intersection['id']]["phase_startLane_mapping"][phase_i] = start_lane
            # lane_phase_info_dict[intersection['id']]["phase_roadLink_mapping"][phase_i] = lane_pair

    return lane_phase_info_dict


def plot_data_lists(data_list,
                    label_list,
                    length=10,
                    height=6,
                    x_label='x',
                    y_label='y',
                    label_fsize=14,
                    save=True,
                    figure_name='temp'):
    '''
    data_list: data1, data2, data3...
    plot this datas in one plot
    '''
    import matplotlib
    if save:
        matplotlib.use('PDF')
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(length, height))
    ax.grid(True)

    for data, label in zip(data_list, label_list):
        ax.plot(data, label=label)  # mec='none', ms=3, label='Algo1 $\\lambda=0.005$'

    ax.plot()
    ax.set_xlabel(x_label, fontsize=label_fsize)
    ax.set_ylabel(y_label, fontsize=label_fsize)
    ax.legend()
    ax.grid(True)

    if save:
        plt.savefig(figure_name)
    else:
        plt.show()


def sample_aug(state):
    state_ = {}
    # state_ = {str(id_): state[str(id_)] for id_ in state}
    for id_ in state:
        temp = [e for e in state[str(id_)][0]]
        tt = temp[3]
        for i in range(3, 0, -1):
            temp[i] = temp[i - 1]
        temp[0] = tt
        temp[4] = (temp[4] % 4) + 1

        ttemp = np.array(temp)
        state_size = len(ttemp.flatten())
        ttemp = np.reshape(ttemp, [1, state_size])
        state_[str(id_)] = ttemp

    return state_

def sample_aug_single(state):

    temp = [e for e in state[0]]
    tt = temp[3]
    for i in range(3, 0, -1):
        temp[i] = temp[i - 1]
    temp[0] = tt
    temp[4] = (temp[4] % 4) + 1

    state_ = np.array(temp)
    state_size = len(state_.flatten())
    state_ = np.reshape(state_, [1, state_size])

    return state_

def sample_aug_timing(state):

    temp = [e for e in state[0]]
    tt = temp[4]
    for i in range(4, 1, -1):
        temp[i] = temp[i - 1]
    temp[1] = tt
    temp[5] = (temp[5] % 4) + 1

    state_ = np.array(temp)
    state_size = len(state_.flatten())
    state_ = np.reshape(state_, [1, state_size])

    return state_

In [ ]:
import math
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from time import *
from copy import deepcopy

# from cityflow_env import CityFlowEnvM
# from utility import *
# # from network import MultiLightAgent
# from dqn_agent import MDQNAgent, DQNAgent

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


def main():
    date = datetime.now().strftime('%Y%m%d_%H%M%S')
    dataset = "1_3"

    cityflow_config = {
        "interval": 1,
        "seed": 0,
        "laneChange": False,
        "dir": "/content/",
        "roadnetFile": "roadnet_1_3.json",
        "flowFile": "syn_1_3_gaussian_500_1h.json",
        # "roadnetFile": "jinan/jinan_roadnet.json",
        # "flowFile": "jinan/jinan_flow.json",
        # "roadnetFile": "hangzhou/hangzhou_roadnet.json",
        # "flowFile": "hangzhou/hangzhou_flow.json",
        "rlTrafficLight": True,
        "saveReplay": False,
        "roadnetLogFile": "replayRoadNet.json",
        "replayLogFile": "replayLogFile.txt"
    }


    with open(os.path.join("/content/", "cityflow.config"), "w") as json_file:
        json.dump(cityflow_config, json_file)

    config = {
        'cityflow_config_file': "/content/cityflow.config",
        'epoch': 200,
        'num_step': 3600,  # 每个epoch的执行步数
        'save_freq': 1,
        'phase_step': 5,  # 每个相位的基础持续时间
        'model': 'DQN',
        'batch_size': 32
    }

    cityflow_config = json.load(open(config['cityflow_config_file']))
    roadnetFile = cityflow_config['dir'] + cityflow_config['roadnetFile']
    config["lane_phase_info"] = parse_roadnet(roadnetFile)

    intersection_id = list(config['lane_phase_info'].keys())  # all intersections
    config["intersection_id"] = intersection_id
    phase_list = {id_: config["lane_phase_info"][id_]["phase"] for id_ in intersection_id}
    config["phase_list"] = phase_list

    timing_list = {id_: [i*5 + config['phase_step'] for i in range(1,5)] for id_ in intersection_id}
    config['timing_list'] = timing_list

    model_dir = "model/{}_{}".format(config['model'], date)
    result_dir = "result/{}_{}".format(config['model'], date)
    config["result_dir"] = result_dir

    if not os.path.exists("model"):
        os.makedirs("model")
    if not os.path.exists("result"):
        os.makedirs("result")
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    env = CityFlowEnvM(config["lane_phase_info"],
                       intersection_id,
                       num_step=config["num_step"],
                       thread_num=8,
                       cityflow_config_file=config["cityflow_config_file"],
                       dataset=dataset
                       )

    config["state_size"] = env.state_size

    Magents = {}
    for id_ in intersection_id:
        agent = DQNAgent(id_,
                         state_size=config["state_size"],
                         action_size=len(phase_list[id_]),
                         batch_size=config["batch_size"],
                         phase_list=phase_list[id_],
                         timing_list=timing_list[id_],
                         env=env)
        Magents[id_] = agent

    EPISODES = config['epoch']
    total_step = 0
    episode_travel_time = []
    with open(result_dir + "/" + "Travel Time-" + dataset + "-IPDALight.csv", 'a+') as ttf:
        ttf.write("travel time\n")
    ttf.close()

    with open(result_dir + "/" + "Greenwave-" + dataset + "-" + ".txt", 'a+') as ttf:
        ttf.write("Greenwave\n")
    ttf.close()

    with tqdm(total=EPISODES * config['num_step']) as pbar:
        for i in range(EPISODES):
            env.reset()
            state = {}
            action = {}
            action_phase = {}
            timing_phase = {}
            reward = {id_: 0 for id_ in intersection_id}
            rest_timing = {id_: 0 for id_ in intersection_id}
            timing_choose = {id_: [] for id_ in intersection_id}
            pressure = {id_: [] for id_ in intersection_id}
            green_wave = {id_: [] for id_ in intersection_id}
            for id_ in intersection_id:
                state[id_] = env.get_state_(id_)

            for episode_length in range(config['num_step']):
                for id_, t in rest_timing.items():
                    if t == 0:
                        if episode_length != 0:
                            # remember and replay the last transition
                            reward[id_] = env.get_reward_(id_)
                            Magents[id_].remember(state[id_], action_phase[id_], reward[id_], next_state[id_])
                            Magents[id_].replay()
                            state[id_] = next_state[id_]

                            # PRESSURE[id_].append(env.get_pressure_(id_))

                        action[id_] = Magents[id_].choose_action(state[id_])
                        action_phase[id_] = phase_list[id_][action[id_]]

                        p, timing_phase[id_] = env.get_timing_(id_, action_phase[id_])
                        rest_timing[id_] = timing_phase[id_]
                        timing_choose[id_].append(timing_phase[id_])
                        pressure[id_].append(p)
                        green_wave[id_].append([action_phase[id_], timing_phase[id_]])


                next_state, reward_ = env.step(action_phase, episode_length)  # one step

                total_step += 1
                pbar.update(1)
                print_reward = deepcopy(reward)
                pbar.set_description(
                    "t_st:{}, epi:{}, st:{}, r:{} ".format(total_step, i + 1, episode_length, print_reward))

                for id_ in rest_timing:
                    rest_timing[id_] -= 1
                # print("rest_timing: ", rest_timing, "\n")

            episode_travel_time.append(env.eng.get_average_travel_time())
            with open(result_dir + "/" + "Travel Time-" + dataset + "-IPDALight.csv", 'a+') as ttf:
                ttf.write("{}\n".format(env.eng.get_average_travel_time()))
            ttf.close()

            with open(result_dir + "/" + "Greenwave-" + dataset + "-" + ".txt", 'a+') as ttf:
                ttf.write("epoch " + str(i + 1) + "\t")
                ttf.write(str(green_wave) + "\n\n")
            ttf.close()

            print('\n')
            print('Epoch {} travel time:'.format(i+1), env.eng.get_average_travel_time())

        df = pd.DataFrame({"travel time": episode_travel_time})
        df.to_csv(result_dir + '/IPDALight.csv', index=False)

        # save figure
        plot_data_lists([episode_travel_time], ['travel time'], figure_name=result_dir + '/travel time.pdf')

        # env.bulk_log()


if __name__ == '__main__':
    start_time = time()
    main()
    end_time = time()
    run_time = end_time - start_time
    print('Run time:', run_time)
# /content/ssss/


  0%|          | 0/720000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 45ms/step


t_st:5, epi:1, st:4, r:{'intersection_1_1': 0, 'intersection_2_1': 0, 'intersection_3_1': 0} :   0%|          | 5/720000 [00:00<68:09:40,  2.93it/s]

1/1 [==============================] - 0s 25ms/step


t_st:10, epi:1, st:9, r:{'intersection_1_1': -0.025, 'intersection_2_1': -0.0401, 'intersection_3_1': -0.037500000000000006} :   0%|          | 10/720000 [00:00<16:55:02, 11.82it/s]

...

t_st:33395, epi:10, st:994, r:{'intersection_1_1': -34.6782, 'intersection_2_1': -7.608400000000003, 'intersection_3_1': -37.3769} :   5%|▍         | 33395/720000 [36:50<14:59:46, 12.72it/s]

1/1 [==============================] - 0s 27ms/step


t_st:33400, epi:10, st:999, r:{'intersection_1_1': -36.9081, 'intersection_2_1': -4.178600000000003, 'intersection_3_1': -34.455499999999994} :   5%|▍         | 33400/720000 [36:51<18:36:24, 10.25it/s]

1/1 [==============================] - 0s 80ms/step


t_st:33403, epi:10, st:1002, r:{'intersection_1_1': -36.9081, 'intersection_2_1': -2.7051999999999996, 'intersection_3_1': -34.455499999999994} :   5%|▍         | 33403/720000 [36:52<18:06:31, 10.53it/s]